# Trouver l'elfe malveillant

Ce notebook t'accompagne pour enquêter dans la base SQLite **`elf_challenge.db`**.
Ton objectif : **identifier l'elfe malveillant**, démontrer la preuve avec **au moins 2 requêtes** et proposer **1 mesure corrective**.

**Tables** : `person`, `gift`, `transactions`, `access_log`.

---
**Conseil** : travaille par hypothèses, note tes trouvailles, et corrèle les logs d'accès avec les transactions autour d'horaires atypiques.


## 1) Préparation de l'environnement

- On va utiliser `sqlite3` et `pandas`.
- La cellule ci-dessous **vérifie** que la base `elf_challenge.db` existe.


In [1]:
import os, sqlite3, random
from datetime import datetime, timedelta

DB_PATH = "elf_challenge.db"

def ensure_db(db_path: str):
    if os.path.exists(db_path):
        print("✔ Base SQLite trouvée :", db_path)
        return
    
    print("⚠ Base absente...", db_path)
   
ensure_db(DB_PATH)

✔ Base SQLite trouvée : elf_challenge.db


## 2) Connexion & aperçu du schéma

In [ ]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(DB_PATH)

# Compter les lignes par table
q = (
    "SELECT 'person' as table_name, COUNT(*) as count FROM person "
    "UNION ALL SELECT 'gift', COUNT(*) FROM gift "
    "UNION ALL SELECT 'transactions', COUNT(*) FROM transactions "
    "UNION ALL SELECT 'access_log', COUNT(*) FROM access_log;"
)
pd.read_sql_query(q, conn)

In [3]:
# Aperçu des colonnes de chaque table (pragma)
tables = ['person','gift','transactions','access_log']
for t in tables:
    df = pd.read_sql_query(f"PRAGMA table_info({t});", conn)
    print(f"\n-- {t} --")
    display(df)


-- person --


,cid,name,type,notnull,dflt_value,pk
0,0,person_id,INTEGER,0,None,1
1,1,full_name,TEXT,1,None,0
2,2,role,TEXT,0,None,0
3,3,hire_date,DATE,0,None,0
4,4,notes,TEXT,0,None,0



-- gift --


,cid,name,type,notnull,dflt_value,pk
0,0,gift_id,INTEGER,0,None,1
1,1,name,TEXT,0,None,0
2,2,status,TEXT,0,None,0
3,3,assigned_to,INTEGER,0,None,0
4,4,created_at,DATETIME,0,None,0



-- transactions --


,cid,name,type,notnull,dflt_value,pk
0,0,tx_id,INTEGER,0,None,1
1,1,gift_id,INTEGER,1,None,0
2,2,actor_id,INTEGER,1,None,0
3,3,action,TEXT,1,None,0
4,4,details,TEXT,0,None,0
5,5,tx_ts,DATETIME,1,None,0



-- access_log --


,cid,name,type,notnull,dflt_value,pk
0,0,log_id,INTEGER,0,None,1
1,1,person_id,INTEGER,0,None,0
2,2,ts,DATETIME,1,None,0
3,3,ip_addr,TEXT,0,None,0
4,4,device,TEXT,0,None,0
5,5,location,TEXT,0,None,0
6,6,action,TEXT,0,None,0
7,7,success,INTEGER,0,None,0


## 3) Échauffement — requêtes simples

1. **Lister** les personnes avec leur rôle.  
2. **Compter** combien de cadeaux par statut (`status`).  
3. **Top** 10 transactions les plus récentes (avec nom de l'acteur).  


In [4]:
# 1) Personnes / rôles
pd.read_sql_query("""
SELECT person_id, full_name, role
FROM person
ORDER BY role, full_name;
""", conn)

,person_id,full_name,role
0,14,E. L. Finkel,elf
1,15,Merry G. Night,elf
2,12,Heloise Bernard,staff
3,11,Marc Leroy,staff
4,16,Service Admin,staff
5,13,Thomas Girard,staff
6,4,Amélie Roy,student
7,9,Camille Lenoir,student
8,1,Clara Dupont,student
9,6,Emma Durand,student


In [5]:
# 2) Cadeaux par statut
pd.read_sql_query("""
SELECT status, COUNT(*) as nb
FROM gift
GROUP BY status
ORDER BY nb DESC;
""", conn)

,status,nb
0,in_stock,33
1,assigned,9
2,missing,5
3,sent,3


In [6]:
# 3) Top 10 transactions récentes
pd.read_sql_query("""
SELECT t.tx_id, t.gift_id, p.full_name AS actor, t.action, t.details, t.tx_ts
FROM transactions t
JOIN person p ON p.person_id = t.actor_id
ORDER BY t.tx_ts DESC
LIMIT 10;
""", conn)

,tx_id,gift_id,actor,action,details,tx_ts
0,23,42,Theo Blanc,note,no defects,2025-11-30 20:59:00
1,177,40,Julien Faure,assign,assign to Merry G. Night,2025-11-30 19:19:00
2,171,24,Camille Lenoir,change_status,sent to student,2025-11-30 15:51:00
3,148,29,Amélie Roy,check_inventory,inventory ok,2025-11-30 15:00:00
4,169,3,Heloise Bernard,change_status,marked missing,2025-11-30 12:20:00
5,61,45,Merry G. Night,assign,assign to Julien Faure,2025-11-30 09:29:00
6,58,50,Lucas Martin,note,no defects,2025-11-29 18:36:00
7,179,32,Amélie Roy,assign,assign to Julien Faure,2025-11-29 16:08:00
8,44,44,Marc Leroy,assign,assign to Amélie Roy,2025-11-29 12:11:00
9,200,33,E. L. Finkel,assign,assign to Amélie Roy,2025-11-28 18:41:00


## 4) Investigation — pistes guidées

### 4.1 Actions nocturnes (00:00–05:00)
- Qui agit la nuit ?
- Y a‑t‑il des **elfes** dans la liste ?


In [7]:
pd.read_sql_query("""
SELECT p.person_id, p.full_name, a.ts, a.action
FROM access_log a
JOIN person p ON p.person_id = a.person_id
WHERE time(a.ts) BETWEEN '00:00:00' AND '05:00:00'
ORDER BY a.ts;
""", conn)

,person_id,full_name,ts,action
0,15,Merry G. Night,2025-11-18 02:13:00,edit_gift
1,11,Marc Leroy,2025-11-18 02:14:00,view_gift
2,15,Merry G. Night,2025-11-20 01:45:00,edit_gift
3,11,Marc Leroy,2025-11-20 01:46:00,view_gift
4,15,Merry G. Night,2025-11-22 03:05:00,edit_gift
5,11,Marc Leroy,2025-11-22 03:06:00,view_gift


### 4.2 Transactions suspectes (emoji 🧝, 'missing')
- Chercher des **indices textuels**.


In [8]:
pd.read_sql_query("""
SELECT t.tx_id, t.gift_id, t.actor_id, p.full_name, t.action, t.details, t.tx_ts
FROM transactions t
JOIN person p ON p.person_id = t.actor_id
WHERE t.details LIKE '%🧝%' OR t.details LIKE '%missing%'
ORDER BY t.tx_ts DESC;
""", conn)

,tx_id,gift_id,actor_id,full_name,action,details,tx_ts
0,169,3,12,Heloise Bernard,change_status,marked missing,2025-11-30 12:20:00
1,205,47,15,Merry G. Night,change_status,marked missing 🧝,2025-11-22 03:05:00
2,206,47,15,Merry G. Night,transfer,moved to secret shelf 🧝,2025-11-22 02:58:00
3,203,36,15,Merry G. Night,change_status,marked missing 🧝,2025-11-20 01:45:00
4,204,36,15,Merry G. Night,transfer,moved to secret shelf 🧝,2025-11-20 01:38:00
5,201,50,15,Merry G. Night,change_status,marked missing 🧝,2025-11-18 02:13:00
6,207,14,15,Merry G. Night,change_status,marked missing 🧝,2025-11-18 02:13:00
7,202,50,15,Merry G. Night,transfer,moved to secret shelf 🧝,2025-11-18 02:06:00
8,208,14,15,Merry G. Night,transfer,moved to secret shelf 🧝,2025-11-18 02:06:00
9,136,27,3,Lucas Martin,change_status,marked missing,2025-11-15 09:41:00


### 4.3 IPs & devices partagés
- Plusieurs comptes depuis la **même IP / device** ?


In [15]:
pd.read_sql_query("""
SELECT ip_addr, device, COUNT(DISTINCT person_id) as nb_accounts,
GROUP_CONCAT(DISTINCT person_id) as person_ids
FROM access_log
GROUP BY ip_addr, device
HAVING nb_accounts > 1
ORDER BY nb_accounts DESC;
""", conn)

,ip_addr,device,nb_accounts,person_ids
0,10.0.0.45,phone-A,10,"8,12,10,13,6,9,1,14,11,2"
1,10.0.0.45,raspberry-pi-3,10,"12,2,14,3,7,13,5,1,15,11"
2,10.0.4.12,raspberry-pi-3,10,"11,4,1,9,16,7,8,13,10,14"
3,10.0.5.9,tablet-7,10,"11,12,16,14,6,7,3,9,10,4"
4,172.16.8.10,phone-A,10,"5,6,12,9,13,8,14,2,3,16"
5,10.0.0.45,laptop-12,9,"5,6,12,11,9,16,15,14,8"
6,10.0.4.12,laptop-12,9,"1,8,7,4,16,15,6,13,9"
7,10.0.5.9,laptop-12,9,"3,16,13,1,7,14,5,6,4"
8,10.0.0.45,tablet-7,8,"9,15,7,16,6,12,3,14"
9,10.0.5.9,desktop-3,8,"2,12,15,16,10,8,11,13"


### 4.4 Corrélation `transactions` ↔ `access_log` (± 2 minutes)
- Valider qu'une action a eu lieu **pendant une session** donnée.


In [10]:
pd.read_sql_query("""
SELECT t.tx_id, t.gift_id, t.actor_id, p.full_name as actor, t.details, t.tx_ts,
       a.log_id, a.person_id, a.ip_addr, a.device, a.location, a.action as access_action, a.ts as access_ts
FROM transactions t
JOIN person p ON p.person_id = t.actor_id
LEFT JOIN access_log a
  ON a.person_id = t.actor_id
 AND ABS(strftime('%s', t.tx_ts) - strftime('%s', a.ts)) <= 120 // < 2 minutes
WHERE t.details LIKE '%missing%'
ORDER BY t.tx_ts DESC
LIMIT 50;
""", conn)

,tx_id,gift_id,actor_id,actor,details,tx_ts,log_id,person_id,ip_addr,device,location,access_action,access_ts
0,169,3,12,Heloise Bernard,marked missing,2025-11-30 12:20:00,NaN,NaN,None,None,None,None,None
1,205,47,15,Merry G. Night,marked missing 🧝,2025-11-22 03:05:00,255.0,15.0,10.0.0.45,raspberry-pi-3,warehouse,edit_gift,2025-11-22 03:05:00
2,203,36,15,Merry G. Night,marked missing 🧝,2025-11-20 01:45:00,253.0,15.0,10.0.0.45,raspberry-pi-3,warehouse,edit_gift,2025-11-20 01:45:00
3,201,50,15,Merry G. Night,marked missing 🧝,2025-11-18 02:13:00,251.0,15.0,10.0.0.45,raspberry-pi-3,warehouse,edit_gift,2025-11-18 02:13:00
4,207,14,15,Merry G. Night,marked missing 🧝,2025-11-18 02:13:00,251.0,15.0,10.0.0.45,raspberry-pi-3,warehouse,edit_gift,2025-11-18 02:13:00
5,136,27,3,Lucas Martin,marked missing,2025-11-15 09:41:00,NaN,NaN,None,None,None,None,None
6,153,16,10,Theo Blanc,marked missing,2025-11-14 09:48:00,NaN,NaN,None,None,None,None,None
7,191,8,1,Clara Dupont,marked missing,2025-11-12 20:50:00,NaN,NaN,None,None,None,None,None
8,91,18,11,Marc Leroy,marked missing,2025-11-09 15:15:00,NaN,NaN,None,None,None,None,None
9,186,30,3,Lucas Martin,marked missing,2025-11-02 20:21:00,NaN,NaN,None,None,None,None,None


Au vu de ce qu'on comprend avec les requêtes ci-dessus, on peut s'interroger sur les activités de Merry G. Night...

## 5) Ton enquête — espace de travail

Utilise les cellules ci‑dessous pour :  
- Formuler tes hypothèses
- Tester tes requêtes
- Consolider la **preuve** (au moins 2 requêtes pertinentes)


In [18]:
# Requête (activité nocturne, 00:00–05:00)

pd.read_sql_query("""
    SELECT a.ts, a.ip_addr, a.device, a.location, a.action
    FROM access_log a
    WHERE a.person_id = 15
    AND time(a.ts) BETWEEN '00:00:00' AND '05:00:00'
    ORDER BY a.ts;
""", conn)

,ts,ip_addr,device,location,action
0,2025-11-18 02:13:00,10.0.0.45,raspberry-pi-3,warehouse,edit_gift
1,2025-11-20 01:45:00,10.0.0.45,raspberry-pi-3,warehouse,edit_gift
2,2025-11-22 03:05:00,10.0.0.45,raspberry-pi-3,warehouse,edit_gift


On regarde les activités de nuit pour Merry G. Night qui `edit_gift` autour de 2025-11-18 02:13, 2025-11-20 01:45, 2025-11-22 03:05, depuis **10.0.0.45** / **raspberry-pi-3** en warehouse — **activité anormale**.

In [20]:
# Requête (corrélation transactions ↔ accès ±120s, contenu suspect)
pd.read_sql_query("""
    SELECT t.tx_id, t.gift_id, t.action, t.details, t.tx_ts, a.ts AS access_ts, a.ip_addr, a.device, a.location
    FROM transactions t
    JOIN access_log a
    ON a.person_id = t.actor_id
    AND ABS(strftime('%s', t.tx_ts) - strftime('%s', a.ts)) <= 120
    WHERE t.actor_id = 15
    AND (t.details LIKE '%🧝%' OR t.details LIKE '%missing%')
    ORDER BY t.tx_ts;
""", conn)

,tx_id,gift_id,action,details,tx_ts,access_ts,ip_addr,device,location
0,201,50,change_status,marked missing 🧝,2025-11-18 02:13:00,2025-11-18 02:13:00,10.0.0.45,raspberry-pi-3,warehouse
1,207,14,change_status,marked missing 🧝,2025-11-18 02:13:00,2025-11-18 02:13:00,10.0.0.45,raspberry-pi-3,warehouse
2,203,36,change_status,marked missing 🧝,2025-11-20 01:45:00,2025-11-20 01:45:00,10.0.0.45,raspberry-pi-3,warehouse
3,205,47,change_status,marked missing 🧝,2025-11-22 03:05:00,2025-11-22 03:05:00,10.0.0.45,raspberry-pi-3,warehouse


Les transactions de **Merry G. Night** 
- `change_status` → *marked missing 🧝*, `transfer` → *moved to secret shelf 🧝* sont **synchronisées** avec ses connexions nocturnes
- même IP/device/lieu → lien direct entre sa session et les cadeaux passés manquants

## 6) Conclusion — Rapport court
**Elfe malveillant identifié** : `person_id = 15`, `full_name = Merry G. Night`

Mesure corrective proposée : 
> Imposer MFA immédiatement pour tous les comptes sensibles, isoler le device `raspberry-pi-3` (warehouse) pour audit forensique et bloquer l’IP `10.0.0.45`, avec une alerte automatique sur toute action entre 00:00 et 05:00